In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [2]:
#Hyperparameters
learning_rate = 0.0005
gamma         = 0.98
lmbda         = 0.95
eps_clip      = 0.1
K_epoch       = 3
T_horizon     = 20
class Agent(nn.Module):
    def __init__(self, state_dim,action_dim,learning_rate,env_num):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.env_num = env_num
        
        super(Agent,self).__init__()
        self.memory = [[] for _ in range(self.env_num)]
        
        
        self.fc1 = nn.Linear(self.state_dim,256)
        self.policy = nn.Linear(256, self.action_dim)
        self.value = nn.Linear(256, 1)
        self.optimizer = optim.Adam(self.parameters(),lr = learning_rate)
        
    def get_action(self,x, softmax_dim = 0):
        x = F.relu(self.fc1(x))
        x = self.policy(x)
        prob = F.softmax(x, dim = softmax_dim)
        return prob
    
    def get_value(self,x):
        x = F.relu(self.fc1(x))
        x = self.value(x)
        return x
    
    def put_data(self,i,data):
        self.memory[i].append(data)
        
    def make_batch(self,i):
        state_list, action_list, reward_list, next_state_list, prob_list, done_list = [],[],[],[],[],[]
        for data in self.memory[i]:
            state,action,reward,next_state,prob,done = data
            state_list.append(state)
            action_list.append([action])
            reward_list.append([reward])
            prob_list.append([prob])
            next_state_list.append(next_state)
            done_mask = 0 if done else 1
            done_list.append([done_mask])
        
        
        s,a,r,next_s,done_mask,prob = torch.tensor(state_list,dtype=torch.float),\
                                        torch.tensor(action_list),torch.tensor(reward_list),\
                                        torch.tensor(next_state_list,dtype=torch.float),\
                                        torch.tensor(done_list,dtype = torch.float),\
                                        torch.tensor(prob_list)
        return s,a,r,next_s,done_mask,prob
    
    def train(self):
        for i in range(K_epoch):
            stop = 0
            for env in range(self.env_num):
                state,action,reward, next_state,done_mask,action_prob = self.make_batch(env)
                #print('state.shape : ',state.shape)
                #print('state.shape[0] : ',state.shape[0])
                if state.shape[0] == 0 :
                    stop += 1
                    continue
                td_error = reward + gamma * self.get_value(next_state) * done_mask
                delta = td_error - self.get_value(state)
                delta = delta.detach().numpy()
                advantage_list = []
                advantage = 0.0
                for delta_t in delta[::-1]:
                    advantage = gamma * lmbda * advantage + delta_t[0]
                    advantage_list.append([advantage])
                advantage_list.reverse()
                if env == stop:
                    all_advantage = torch.tensor(advantage_list,dtype = torch.float)
                    all_action = action
                    all_state = state
                    all_action_prob = action_prob
                    all_td_error = td_error
                else :
                    all_advantage = torch.cat((all_advantage,torch.tensor(advantage_list,dtype= torch.float)))
                    all_action = torch.cat((all_action,action))
                    all_state = torch.cat((all_state,state))
                    all_action_prob = torch.cat((all_action_prob,action_prob))
                    all_td_error = torch.cat((all_td_error,td_error))
            #print('all_state : ',all_state.shape)
            now_action = self.get_action(all_state,softmax_dim = 1)
            now_action = now_action.gather(1,all_action)
            
            ratio = torch.exp(torch.log(now_action) - torch.log(all_action_prob))
            
            surr1 = ratio * all_advantage
            surr2 = torch.clamp(ratio , 1-eps_clip, 1 + eps_clip) * all_advantage
            loss = - torch.min(surr1,surr2) + F.smooth_l1_loss(self.get_value(all_state),all_td_error.detach())
            
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()
        self.memory = [[] for _ in range(self.env_num)]

In [3]:
import numpy as np
import os
from Building import Building
#from Agent import Agent
import time
#====================================================================================


#====================================================================================
#Building Setting
lift_num = 1
buliding_height = 5
max_people_in_floor = 8
max_people_in_elevator = 10

add_people_at_step = 25
add_people_prob = 0.8

#Create building with 4 elevators, height 10, max people 30 in each floor

env_num = 4

#building = Building(lift_num, buliding_height, max_people_in_floor,max_people_in_elevator)

#Agent controls each elevator
#agent = Agent(buliding_height, lift_num, 4)
#agent.reload(280)
#The goal is to bring down all the people in the building to the ground floor

epochs = 1000
max_steps = 100
global_step = 0
T_horizon = 20
reward_list = []
print_interval = 20

def finish_checker(x):
    return x[:-2]

In [4]:
model = Agent((buliding_height)+ max_people_in_elevator + (lift_num *2),4,learning_rate,env_num)
print_interval = 20
ave_reward = 0

In [5]:
building_list = [Building(lift_num, buliding_height, max_people_in_floor,max_people_in_elevator) for _ in range(env_num)]

In [6]:
epochs = 10001
print_interval = 20

In [7]:
for epoch in range(epochs):
    [building.empty_building() for building in building_list]
    [building.generate_people(add_people_prob) for building in building_list]
    state_list = [building.get_state() for building in building_list]
    
    done_list = [False for _ in range(env_num)]
    global_step = 0
    while not all(done_list):
        #print('global_step : ', global_step,done_list)
        for t in range(T_horizon):
            #print('global_step : ',global_step,' done_list : ',done_list)
            global_step += 1
            if (global_step % 25 == 0) & global_step > 0 :
                #building.generate_people(add_people_prob/2)
                pass
            action_prob_list = [model.get_action(torch.from_numpy(np.array(state)).float()) for state in state_list]
            m_list = [Categorical(action_prob) for action_prob in action_prob_list]
            action_list = [m.sample().item() for m in m_list]
            [building.perform_action([action_list[idx]]) for idx,building in enumerate(building_list)]
            reward_list = [building.get_reward() for building in building_list]
            
            next_state_list = [building.get_state() for building in building_list]
            finished_list = [next_state.copy() for next_state in next_state_list]
            finished_list = list(map(finish_checker, finished_list))
            for i in range(env_num):
                if (sum(finished_list[i]) == 0.0) : #and (done_list[i] != True):
                    reward_list[i] = 100. #* building.target
                    done_list[i] = True
            #print(sum(finished))
            #print('global_step : ',global_step,'state : ',state, 'action : ', action, 'reward : ',reward/float(first_state), 'done : ',done)
            #print('global_step : ',global_step,'state : ',state, 'action : ', action, 'reward : ',reward/10., 'done : ',done)
            #model.put_data((state, action, reward/float(first_state), next_state, action_prob[action].item(), done))
            for i in range(env_num):
                #if reward_list[i] == 0 and done_list[i] == True:
                #    continue
                model.put_data(i,(state_list[i], action_list[i], reward_list[i]/100.0,\
                                next_state_list[i], action_prob_list[i][action_list[i]].item(), done_list[i]))
            state_list = next_state_list
            for i in range(env_num):
                if (global_step > 300):
                    done_list[i] = True
            if all(done_list) == True:
                break
        model.train()
    ave_reward += global_step 
    #print("Epoch: %d Step: %d Average Reward: %.4f"%(epoch, global_step, ave_reward/global_step))
    if epoch%print_interval==0 and epoch!=0:
        print("# of episode :{}, avg score : {:.1f}".format(epoch, ave_reward/print_interval))
        ave_reward = 0
    if (epoch % 100 == 0 )& (epoch != 0):
        torch.save(model.state_dict(), './model_weights/vari_env_model_'+str(epoch))
    reward_list.append(global_step)

# of episode :20, avg score : 272.6
# of episode :40, avg score : 164.8
# of episode :60, avg score : 129.4
# of episode :80, avg score : 105.2


KeyboardInterrupt: 